<a href="https://colab.research.google.com/github/tendiek/unsupervised-predict-streamlit-template/blob/master/Unsupervised_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

import operator 
import heapq 

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')


In [3]:
sample_df = pd.read_csv("sample_submission.csv")
sample_df.head

<bound method NDFrame.head of                   Id  rating
0             1_2011     1.0
1             1_4144     1.0
2             1_5767     1.0
3             1_6711     1.0
4             1_7318     1.0
...              ...     ...
5000014  162541_4079     1.0
5000015  162541_4467     1.0
5000016  162541_4980     1.0
5000017  162541_5689     1.0
5000018  162541_7153     1.0

[5000019 rows x 2 columns]>

In [4]:
train_df = pd.read_csv("train.csv")
train_df.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [5]:
test_df = pd.read_csv("test.csv")
test_df.head()

,userId,movieId
0,5,788
1,68,7438
2,336,40412
3,803,3822
4,547,903


In [6]:
movie_df = pd.read_csv("movies.csv")
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
train_df.shape

(10000038, 4)

In [8]:
train_df.drop('timestamp', inplace=True, axis=1)
train_df.head()

,userId,movieId,rating
0,5163,57669,4.0
1,106343,5,4.5
2,146790,5459,5.0
3,106362,32296,2.0
4,9041,366,3.0


In [9]:
sampled_train = train_df.sample(n=50000)
sampled_train.head()

,userId,movieId,rating
5470153,83168,55232,2.5
2358029,61702,290,5.0
9793172,153815,208,1.0
1469448,2465,1500,4.5
5691293,134104,1584,3.0


In [10]:
comb_movie = pd.merge(sampled_train, movie_df, left_on="movieId", right_on="movieId", how="left")
comb_movie.head()

,userId,movieId,rating,title,genres
0,83168,55232,2.5,Resident Evil: Extinction (2007),Action|Horror|Sci-Fi|Thriller
1,61702,290,5.0,Once Were Warriors (1994),Crime|Drama
2,153815,208,1.0,Waterworld (1995),Action|Adventure|Sci-Fi
3,2465,1500,4.5,Grosse Pointe Blank (1997),Comedy|Crime|Romance
4,134104,1584,3.0,Contact (1997),Drama|Sci-Fi


In [11]:
util_matrix = sampled_train.pivot_table(index=['userId'],columns=['movieId'],values='rating') 
util_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,203787,203928,204178,204376,204698,205325,205425,205573,207291,207892
userId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
util_matrix_norm = util_matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
util_matrix_norm.fillna(0, inplace=True)
util_matrix_norm = util_matrix_norm.T
util_matrix_norm = util_matrix_norm.loc[:, (util_matrix_norm != 0).any(axis=0)]
util_matrix_sparse = sp.sparse.csr_matrix(util_matrix_norm.values)
util_matrix_sparse

<7706x7644 sparse matrix of type '<class 'numpy.float64'>'
	with 20759 stored elements in Compressed Sparse Row format>

In [13]:
util_matrix_norm

userId,9,12,31,72,80,84,98,175,187,247,...,162387,162424,162445,162481,162484,162486,162495,162508,162516,162534
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
user_similarity = cosine_similarity(util_matrix_sparse.T) 
user_sim_df = pd.DataFrame(user_similarity, 
                           index = util_matrix_norm.columns, 
                           columns = util_matrix_norm.columns)
user_sim_df[:5]

userId,9,12,31,72,80,84,98,175,187,247,...,162387,162424,162445,162481,162484,162486,162495,162508,162516,162534
userId,,,,,,,,,,,,,,,,,,,,,
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def collab_generate_top_N_recommendations(reference_user, N=10, k=20): 
    if reference_user not in user_sim_df.columns:
        return comb_movie.groupby(['movieId', 'title'] ).mean().sort_values(by='rating',
                                        ascending=False).index[:N].to_list()
    
    sim_users = user_sim_df.sort_values(by=reference_user, ascending=False).index[1:k+1]
    favorite_user_items = []  
    most_common_favorites = {} 
    
    for i in sim_users:
        max_score = util_matrix_norm.loc[:, i].max()  
        favorite_user_items.append(util_matrix_norm[util_matrix_norm.loc[:, i]==max_score].index.tolist())
        
    for item_collection in range(len(favorite_user_items)):
        for item in favorite_user_items[item_collection]: 
            if item in most_common_favorites:
                most_common_favorites[item] += 1
            else:
                most_common_favorites[item] = 1

    sorted_list = sorted(most_common_favorites.items(), key=operator.itemgetter(1), reverse=True)[:N]
    top_N = [x[0] for x in sorted_list]
    return top_N  

In [18]:
collab_generate_top_N_recommendations(12)

[230, 7262, 3091, 2313, 3505, 3730, 1025, 47099, 5948, 7076]

In [19]:
def movie_rating(reference_movie, reference_user, k=20, threshold=0.0):
    sim_users = user_sim_df.sort_values(by=reference_user, ascending=False).index[1:k+1]
    user_values = user_sim_df.sort_values(by=reference_user, ascending=False).loc[:,reference_user].tolist()[1:k+1]
    rating_list = [] 
    weight_list = []
    
    for sim_idx, user_id in enumerate(sim_users):
        rating = util_matrix.loc[user_id, reference_movie]
        similarity = user_values[sim_idx]
        if (np.isnan(rating)) or (similarity < threshold):
            continue
        elif not np.isnan(rating):
            rating_list.append(rating*similarity)
            weight_list.append(similarity)
    try:
        predicted_rating = sum(rating_list)/sum(weight_list) 
    except ZeroDivisionError:  
        predicted_rating = np.mean(util_matrix[reference_movie])
    return predicted_rating

In [20]:
y = sampled_train['rating']

X = sampled_train.drop('rating', axis=1)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [23]:
knn=KNeighborsRegressor(metric='cosine', n_neighbors=10)

In [24]:
knn.fit(X,y)

KNeighborsRegressor(metric='cosine', n_neighbors=10)

In [25]:
y_predicted = knn.predict(X_test)


In [33]:

df_predictions = pd.DataFrame(y_predicted, columns=['rating'])
df_predictions ['Id'] = test_df['userId'].apply(str) +"_" + test_df['movieId'].apply(str)
titles= list(df_predictions.columns)
titles[0],titles[1]=titles[1],titles[0]
df_predictions = df_predictions[titles]
df_predictions

,Id,rating
0,5_788,3.70
1,68_7438,3.40
2,336_40412,3.95
3,803_3822,3.65
4,547_903,3.20
...,...,...
9995,983_3173,3.90
9996,794_3984,3.35
9997,72_2146,3.25
9998,886_93721,3.70


In [34]:
file_name = "Tendani_submission_1.csv"
df_predictions.to_csv(file_name, index = False)